# 6. Topic extraction from NER

## Setup

In [1]:
%run __init__.py

In [12]:
from bokeh.io import output_notebook

output_notebook()

Loading BokehJS ...

In [2]:
import pandas as pd

PMC_FILE_PATH = os.path.join(NOTEBOOK_2_RESULTS_DIR, 'pmc_dataframe.pkl')

pmc_df = pd.read_pickle(PMC_FILE_PATH)
publications = pmc_df['text_cleaned'].values

/home/alejandro/.envs/edma/lib/python3.7/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


## Loading the NER model

In [3]:
from src.utils import load_object

ner = load_object(os.path.join(NOTEBOOK_4_RESULTS_DIR, 'ner_system.pkl'))

## Testing the system

In [16]:
text = publications[-2]

### Entity linking

In [17]:
import en_core_sci_lg
from collections import Counter

nlp = en_core_sci_lg.load()
entities = ner.transform([text])
entities[0][:10]

[['fungus',
  'Alternaria brassicicola',
  'seed transmission',
  'fungal',
  'impact',
  'studies',
  'Arabidopsis thaliana',
  'A. brassicicola',
  'level',
  'susceptibility',
  'water stress',
  'ability',
  'mutants',
  'MAP kinase AbHog1',
  'seeds',
  'Iacomi-Vasilescu',
  'functional',
  'efficient',
  'infection',
  'dehydrin-like proteins',
  'seed colonization',
  'fungi',
  'exposed to',
  'water',
  'osmotic',
  'stresses',
  'transmission',
  'plants',
  'desiccation',
  'structural',
  'induced',
  'proteins',
  'mechanisms',
  'synthesis',
  'polyols',
  'yeast',
  'glycerol',
  'intracellular',
  'accumulation',
  'hydrophilins',
  'Hydrophilins',
  'hydrophilic',
  'Sa.',
  'genes',
  'expression',
  'deletion',
  'capacity',
  'cellular',
  'filamentous fungi',
  'dehydrins',
  'oxidative',
  'AbHog1',
  'exposure to',
  'mutant',
  'affected',
  'aggressiveness',
  'leaves',
  'reduced',
  'siliques',
  'seed infection',
  'study',
  'data',
  'stress',
  'treatment

In [18]:
from src.entity_linking import WikidataEntityLinker

linker = WikidataEntityLinker()
linked_entities = linker.fit_transform(entities)
linked_entities[0][:5]

[('fungus', 'http://www.wikidata.org/entity/Q764'),
 ('Alternaria brassicicola', 'http://www.wikidata.org/entity/Q5234097'),
 ('seed transmission', 'http://www.wikidata.org/entity/Q21131920'),
 ('fungal', 'http://www.wikidata.org/entity/Q464067'),
 ('impact', 'http://www.wikidata.org/entity/Q977252')]

### Building the graph

In [19]:
from src.graph import WikidataGraphBuilder

graph_builder = WikidataGraphBuilder(max_hops=2)
entity_graph = graph_builder.build_graph(linked_entities[0])

Visiting entity 'Q764' - Curr hop: 0
Visiting entity 'Q16521' - Curr hop: 1
Visiting entity 'Q16889133' - Curr hop: 2
Visiting entity 'Q19478619' - Curr hop: 3
Visiting entity 'Q5127848' - Curr hop: 3
Visiting entity 'Q324254' - Curr hop: 3
Visiting entity 'Q7095059' - Curr hop: 3
Visiting entity 'Q5962346' - Curr hop: 3
Visiting entity 'Q16334298' - Curr hop: 2
Visiting entity 'Q61961344' - Curr hop: 3
Visiting entity 'Q517966' - Curr hop: 3
Visiting entity 'Q7239' - Curr hop: 3
Visiting entity 'Q15059690' - Curr hop: 2
Visiting entity 'Q4167836' - Curr hop: 3
Visiting entity 'Q16521' - Curr hop: 3
Visiting entity 'Q24017414' - Curr hop: 2
Visiting entity 'Q23959932' - Curr hop: 3
Visiting entity 'Q24017465' - Curr hop: 3
Visiting entity 'Q1456955' - Curr hop: 1
Visiting entity 'Q764' - Curr hop: 2
Visiting entity 'Q16521' - Curr hop: 3
Visiting entity 'Q1456955' - Curr hop: 3
Visiting entity 'Q83093' - Curr hop: 3
Visiting entity 'Q326228' - Curr hop: 3
Visiting entity 'Q7175' - Curr

Visiting entity 'Q12136' - Curr hop: 2
Visiting entity 'Q2057971' - Curr hop: 3
Visiting entity 'Q567555' - Curr hop: 3
Visiting entity 'Q814207' - Curr hop: 3
Visiting entity 'Q179630' - Curr hop: 3
Visiting entity 'Q9789161' - Curr hop: 3
Visiting entity 'Q7208' - Curr hop: 3
Visiting entity 'Q42417296' - Curr hop: 2
Visiting entity 'Q12136' - Curr hop: 3
Visiting entity 'Q25383952' - Curr hop: 3
Visiting entity 'Q6177900' - Curr hop: 2
Visiting entity 'Q4167836' - Curr hop: 3
Visiting entity 'Q18123741' - Curr hop: 3
Visiting entity 'Q788926' - Curr hop: 2
Visiting entity 'Q8977284' - Curr hop: 3
Visiting entity 'Q930752' - Curr hop: 3
Visiting entity 'Q4167410' - Curr hop: 3
Visiting entity 'Q18123741' - Curr hop: 3
Visiting entity 'Q12136' - Curr hop: 2
Visiting entity 'Q2057971' - Curr hop: 3
Visiting entity 'Q567555' - Curr hop: 3
Visiting entity 'Q814207' - Curr hop: 3
Visiting entity 'Q179630' - Curr hop: 3
Visiting entity 'Q9789161' - Curr hop: 3
Visiting entity 'Q7208' - Cur

Visiting entity 'Q4167836' - Curr hop: 2
Visiting entity 'Q158695' - Curr hop: 2
Visiting entity 'Q16521' - Curr hop: 3
Visiting entity 'Q213907' - Curr hop: 3
Visiting entity 'Q15099763' - Curr hop: 3
Visiting entity 'Q1046315' - Curr hop: 0
Visiting entity 'Q7184903' - Curr hop: 1
Visiting entity 'Q488383' - Curr hop: 2
Visiting entity 'Q35120' - Curr hop: 3
Visiting entity 'Q41798914' - Curr hop: 3
Visiting entity 'Q6576895' - Curr hop: 3
Visiting entity 'Q33104279' - Curr hop: 3
Visiting entity 'Q18205125' - Curr hop: 2
Visiting entity 'Q151885' - Curr hop: 3
Visiting entity 'Q167312' - Curr hop: 3
Visiting entity 'Q29431432' - Curr hop: 3
Visiting entity 'Q4406616' - Curr hop: 3
Visiting entity 'Q7184903' - Curr hop: 3
Visiting entity 'Q64920589' - Curr hop: 3
Visiting entity 'Q5159081' - Curr hop: 3
Visiting entity 'Q23958852' - Curr hop: 2
Visiting entity 'Q23960977' - Curr hop: 3
Visiting entity 'Q19478619' - Curr hop: 3
Visiting entity 'Q23958852' - Curr hop: 3
Visiting entity

Visiting entity 'Q21191270' - Curr hop: 2
Visiting entity 'Q1983062' - Curr hop: 3
Visiting entity 'Q2431196' - Curr hop: 3
Visiting entity 'Q15416' - Curr hop: 3
Visiting entity 'Q5398426' - Curr hop: 3
Visiting entity 'Q3464665' - Curr hop: 3
Visiting entity 'Q7453224' - Curr hop: 3
Visiting entity 'Q619474' - Curr hop: 2
Visiting entity 'Q1610808' - Curr hop: 3
Visiting entity 'Q22342076' - Curr hop: 3
Visiting entity 'Q22342077' - Curr hop: 3
Visiting entity 'Q22342078' - Curr hop: 3
Visiting entity 'Q22342079' - Curr hop: 3
Visiting entity 'Q22342080' - Curr hop: 3
Visiting entity 'Q22342081' - Curr hop: 3
Visiting entity 'Q28225717' - Curr hop: 3
Visiting entity 'Q22342079' - Curr hop: 1
Visiting entity 'Q21191270' - Curr hop: 2
Visiting entity 'Q1983062' - Curr hop: 3
Visiting entity 'Q2431196' - Curr hop: 3
Visiting entity 'Q15416' - Curr hop: 3
Visiting entity 'Q5398426' - Curr hop: 3
Visiting entity 'Q3464665' - Curr hop: 3
Visiting entity 'Q7453224' - Curr hop: 3
Visiting en

Visiting entity 'Q1034411' - Curr hop: 2
Visiting entity 'Q1207505' - Curr hop: 3
Visiting entity 'Q21013863' - Curr hop: 3
Visiting entity 'Q1296024' - Curr hop: 2
Visiting entity 'Q1207505' - Curr hop: 3
Visiting entity 'Q21013863' - Curr hop: 3
Visiting entity 'Q1207505' - Curr hop: 2
Visiting entity 'Q33104279' - Curr hop: 3
Visiting entity 'Q25271255' - Curr hop: 3
Visiting entity 'Q937228' - Curr hop: 3
Visiting entity 'Q29431432' - Curr hop: 2
Visiting entity 'Q16887380' - Curr hop: 3
Visiting entity 'Q134830' - Curr hop: 3
Visiting entity 'Q18123741' - Curr hop: 0
Visiting entity 'Q12136' - Curr hop: 1
Visiting entity 'Q2057971' - Curr hop: 2
Visiting entity 'Q7189713' - Curr hop: 3
Visiting entity 'Q621184' - Curr hop: 3
Visiting entity 'Q19829125' - Curr hop: 3
Visiting entity 'Q639907' - Curr hop: 3
Visiting entity 'Q9700466' - Curr hop: 3
Visiting entity 'Q567555' - Curr hop: 2
Visiting entity 'Q849149' - Curr hop: 3
Visiting entity 'Q814207' - Curr hop: 2
Visiting entity '

...

Visiting entity 'Q6165507' - Curr hop: 3
Visiting entity 'Q5082128' - Curr hop: 3
Visiting entity 'Q68' - Curr hop: 3
Visiting entity 'Q7333880' - Curr hop: 1
Visiting entity 'Q4167836' - Curr hop: 2
Visiting entity 'Q162768' - Curr hop: 2
Visiting entity 'Q155972' - Curr hop: 3
Visiting entity 'Q7333880' - Curr hop: 3
Visiting entity 'Q101965' - Curr hop: 0
Visiting entity 'Q6479193' - Curr hop: 1
Visiting entity 'Q101965' - Curr hop: 2
Visiting entity 'Q6479193' - Curr hop: 3
Visiting entity 'Q1003030' - Curr hop: 3
Visiting entity 'Q3331334' - Curr hop: 3
Visiting entity 'Q367158' - Curr hop: 3
Visiting entity 'Q4167836' - Curr hop: 2
Visiting entity 'Q1003030' - Curr hop: 1
Visiting entity 'Q3077373' - Curr hop: 2
Visiting entity 'Q1799072' - Curr hop: 3
Visiting entity 'Q791801' - Curr hop: 2
Visiting entity 'Q37112258' - Curr hop: 3
Visiting entity 'Q2695280' - Curr hop: 3
Visiting entity 'Q10843872' - Curr hop: 3
Visiting entity 'Q741332' - Curr hop: 2
Visiting entity 'Q61788060

Visiting entity 'Q9147666' - Curr hop: 3
Visiting entity 'Q3065932' - Curr hop: 3
Visiting entity 'Q1441305' - Curr hop: 3
Visiting entity 'Q12136' - Curr hop: 3
Visiting entity 'Q7867' - Curr hop: 3
Visiting entity 'Q9418' - Curr hop: 3
Visiting entity 'Q2903417' - Curr hop: 2
Visiting entity 'Q2057971' - Curr hop: 3
Visiting entity 'Q170082' - Curr hop: 3
Visiting entity 'Q30085742' - Curr hop: 3
Visiting entity 'Q9895080' - Curr hop: 3
Visiting entity 'Q12136' - Curr hop: 2
Visiting entity 'Q2057971' - Curr hop: 3
Visiting entity 'Q567555' - Curr hop: 3
Visiting entity 'Q814207' - Curr hop: 3
Visiting entity 'Q179630' - Curr hop: 3
Visiting entity 'Q9789161' - Curr hop: 3
Visiting entity 'Q7208' - Curr hop: 3
Visiting entity 'Q929833' - Curr hop: 2
Visiting entity 'Q6853484' - Curr hop: 3
Visiting entity 'Q12136' - Curr hop: 3
Visiting entity 'Q54932064' - Curr hop: 3
Visiting entity 'Q170304' - Curr hop: 1
Visiting entity 'Q11173' - Curr hop: 2
Visiting entity 'Q578779' - Curr hop:

Visiting entity 'Q483394' - Curr hop: 3
Visiting entity 'Q8594210' - Curr hop: 3
Visiting entity 'Q773668' - Curr hop: 1
Visiting entity 'Q737498' - Curr hop: 2
Visiting entity 'Q18918145' - Curr hop: 3
Visiting entity 'Q20821071' - Curr hop: 3
Visiting entity 'Q685935' - Curr hop: 3
Visiting entity 'Q21114848' - Curr hop: 3
Visiting entity 'Q1327300' - Curr hop: 2
Visiting entity 'Q1714118' - Curr hop: 3
Visiting entity 'Q41298' - Curr hop: 3
Visiting entity 'Q232932' - Curr hop: 2
Visiting entity 'Q8700963' - Curr hop: 3
Visiting entity 'Q595523' - Curr hop: 3
Visiting entity 'Q1156854' - Curr hop: 3
Visiting entity 'Q3533467' - Curr hop: 3
Visiting entity 'Q24707952' - Curr hop: 3
Visiting entity 'Q9068936' - Curr hop: 2
Visiting entity 'Q4167836' - Curr hop: 3
Visiting entity 'Q773668' - Curr hop: 3
Visiting entity 'Q1293664' - Curr hop: 2
Visiting entity 'Q8903592' - Curr hop: 3
Visiting entity 'Q1260632' - Curr hop: 3
Visiting entity 'Q17537576' - Curr hop: 3
Visiting entity 'Q11

Visiting entity 'Q635115' - Curr hop: 3
Visiting entity 'Q28793' - Curr hop: 2
Visiting entity 'Q8451881' - Curr hop: 3
Visiting entity 'Q11862829' - Curr hop: 3
Visiting entity 'Q4671286' - Curr hop: 3
Visiting entity 'Q1660187' - Curr hop: 3
Visiting entity 'Q1660187' - Curr hop: 3
Visiting entity 'Q1660187' - Curr hop: 2
Visiting entity 'Q165650' - Curr hop: 3
Visiting entity 'Q8953936' - Curr hop: 3
Visiting entity 'Q11862829' - Curr hop: 3
Visiting entity 'Q4671286' - Curr hop: 3
Visiting entity 'Q28793' - Curr hop: 3
Visiting entity 'Q4482' - Curr hop: 3
Visiting entity 'Q9292847' - Curr hop: 3
Visiting entity 'Q11424' - Curr hop: 3
Visiting entity 'Q183' - Curr hop: 0
Visiting entity 'Q1410828' - Curr hop: 1
Visiting entity 'Q183' - Curr hop: 2
Visiting entity 'Q1410828' - Curr hop: 3
Visiting entity 'Q3624078' - Curr hop: 3
Visiting entity 'Q6256' - Curr hop: 3
Visiting entity 'Q43702' - Curr hop: 3
Visiting entity 'Q7270' - Curr hop: 3
Visiting entity 'Q619610' - Curr hop: 3
V

Visiting entity 'Q1410828' - Curr hop: 3
Visiting entity 'Q3624078' - Curr hop: 3
Visiting entity 'Q6256' - Curr hop: 3
Visiting entity 'Q43702' - Curr hop: 3
Visiting entity 'Q7270' - Curr hop: 3
Visiting entity 'Q619610' - Curr hop: 3
Visiting entity 'Q4209223' - Curr hop: 3
Visiting entity 'Q56061' - Curr hop: 3
Visiting entity 'Q63791824' - Curr hop: 3
Visiting entity 'Q27496' - Curr hop: 3
Visiting entity 'Q27509' - Curr hop: 3
Visiting entity 'Q32' - Curr hop: 2
Visiting entity 'Q4368335' - Curr hop: 3
Visiting entity 'Q3624078' - Curr hop: 3
Visiting entity 'Q6256' - Curr hop: 3
Visiting entity 'Q179164' - Curr hop: 3
Visiting entity 'Q165116' - Curr hop: 3
Visiting entity 'Q123480' - Curr hop: 3
Visiting entity 'Q235' - Curr hop: 2
Visiting entity 'Q3624078' - Curr hop: 3
Visiting entity 'Q179164' - Curr hop: 3
Visiting entity 'Q208500' - Curr hop: 3
Visiting entity 'Q6256' - Curr hop: 3
Visiting entity 'Q133442' - Curr hop: 3
Visiting entity 'Q51576574' - Curr hop: 3
Visiting 

In [20]:
from bokeh.io import show
from bokeh.layouts import gridplot

from src.graph import build_graph_plot

plot = build_graph_plot(entity_graph, f"Linked entities graph")
show(plot)

In [21]:
from src.graph import get_largest_connected_subgraph

connected_entity_subgraph = get_largest_connected_subgraph(entity_graph)

plot = build_graph_plot(connected_entity_subgraph, f"Linked entities graph")
show(plot)

In [25]:
import networkx.algorithms as nxa

from src.graph import get_centrality_algorithm_results

def try_centrality_algorithms(g, algorithms, stop_uris, top_n=9):
    for (algorithm, name) in algorithms:
        print(f'Algorithm: {name}')
        result = get_centrality_algorithm_results(g, algorithm, stop_uris, top_n)
        print(f"Topics:", result)
        print()
        
algorithms = [
    (nxa.centrality.information_centrality, "Information centrality"),
    (nxa.centrality.eigenvector_centrality_numpy, "Eigenvector centrality"),
    (nxa.centrality.closeness_centrality, "Closeness centrality"),
    (nxa.centrality.betweenness_centrality, "Betweenness centrality"),
    (nxa.centrality.load_centrality, "Load centrality")
]

try_centrality_algorithms(connected_entity_subgraph,
                          algorithms,
                          ['Q4167836', 'Q11862829', 'Q13442814',
                           'Q17339814', 'Q24017414', 'Q47154513'])

Algorithm: Information centrality
Topic: [('biological process', 0.00039625063004530603), ('molecular function', 0.0003923114131165757), ('amino acid', 0.0003862031128880044), ('chemical compound', 0.00038483331253055824), ('hydron', 0.00038284173898896613), ('oxygen', 0.0003681937464155676), ('taxon', 0.0003675427221699843), ('chemistry', 0.00036753189831848583), ('carbon', 0.00036738089538130057)]

Algorithm: Eigenvector centrality
Topic: [('molecular function', 0.4234447737728409), ('biological process', 0.24619044733819703), ('hydron', 0.17076392766639165), ('amino acid', 0.13189535962046367), ('glycine dehydrogenase activity', 0.10939691433418723), ('strombine dehydrogenase activity', 0.10555911689242195), ('glycine reductase activity', 0.10345629106452091), ('N-benzyloxycarbonylglycine hydrolase activity', 0.10227781121812288), ('sarcosine dehydrogenase activity', 0.10029626311935241)]

Algorithm: Closeness centrality
Topic: [('chemical compound', 0.22202339986235375), ('chemistr

### Obtaining the topics

## Setting up the pipeline